# Mapa de Earthquake o Terremoto

Usando dataset ../../Data/02ParaLimpiar/02desastres_paralimpiar.csv

## Objetivo

## Mapa de ubicaciones de terremotos con Geopy

## Importar librerías

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 40)
import matplotlib.pyplot as plt
import seaborn as sns
import folium

## Cargar datos

In [2]:
df = pd.read_csv('../../Data/02ParaLimpiar/02desastres_paralimpiar.csv', encoding='utf-8', delimiter=';', engine='python')

# Verificar carga de dataset

##  Resumen básico shape, info, head

In [3]:
df.shape
# Resultado: 16636 filas y 26 columnas)

(16636, 26)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16636 entries, 0 to 16635
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Dis No             16636 non-null  object 
 1   Year               16636 non-null  int64  
 2   Seq                16636 non-null  int64  
 3   Disaster Subgroup  16636 non-null  object 
 4   Disaster Type      16636 non-null  object 
 5   Disaster Subtype   13313 non-null  object 
 6   Country            16636 non-null  object 
 7   ISO                16636 non-null  object 
 8   Region             16636 non-null  object 
 9   Continent          16636 non-null  object 
 10  Location           14825 non-null  object 
 11  Origin             4085 non-null   object 
 12  Associated Dis     3593 non-null   object 
 13  Dis Mag Value      5064 non-null   float64
 14  Dis Mag Scale      15416 non-null  object 
 15  Latitude           2775 non-null   object 
 16  Longitude          277

In [5]:
df.head(2)

,Dis No,Year,Seq,Disaster Subgroup,Disaster Type,Disaster Subtype,Country,ISO,Region,Continent,...,Longitude,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,Total Affected,Total Damages Adj
0,1900-9002-CPV,1900,9002,Climatological,Drought,Drought,Cabo Verde,CPV,Western Africa,Africa,...,NaN,1900,NaN,NaN,1900,NaN,NaN,11000.0,NaN,NaN
1,1900-9001-IND,1900,9001,Climatological,Drought,Drought,India,IND,Southern Asia,Asia,...,NaN,1900,NaN,NaN,1900,NaN,NaN,1250000.0,NaN,NaN


# Earthquake filter

### Filtro principal para Earthquake
Selecciona solo las filas en el DataFrame donde la columna 'Disaster Type' es 'Earthquake'.

In [6]:
# Filtro principal para Earthquake
earthquake_filter = (df['Disaster Type'] == 'Earthquake')
earthquake_filter

0        False
1        False
2         True
3        False
4        False
         ...  
16631    False
16632    False
16633    False
16634    False
16635    False
Name: Disaster Type, Length: 16636, dtype: bool

### Categorías de condiciones asociadas al filtro Earthquake: geográficas, magnitud, geolocalización, temporales, etc.

In [7]:
# Categorías geográficas de Earthquake
earthquake_geographical_data = df[earthquake_filter][['Location', 'ISO', 'Country', 'Region', 'Continent']]

# Categorías de magnitud de Earthquake
earthquake_magnitude_data = df[earthquake_filter][['Dis Mag Scale', 'Dis Mag Value']]

# Categorías de geolocalización de Earthquake
earthquake_geolocation_data = df[earthquake_filter][['Latitude', 'Longitude']]

# Categorías de fecha de Earthquake
earthquake_datetime_data = df[earthquake_filter][['Start Month', 'Start Year', 'End Year', 'End Year']]

# Filtros extras para otras categorías relacionadas con Earthquake
# Considerar y activar solo en caso necesario, cambiando nombre Category por otro

# Mapa de terremotos por geolocalización usando Folium

In [8]:
# Esta línea recupera los datos geográficos relevantes para los terremotos.
# Selecciona las columnas geográficas filtradas usando el indexador loc.
# La llamada reset_index(drop=True) se usa para restablecer el índice del DataFrame resultante y descartar el índice original.
# reset_index garantiza que el índice comience desde 0 y sea continuo sin espacios.
earthquake_geographical_data = df.loc[earthquake_filter, ['Location', 'ISO', 'Country', 'Region', 'Continent']].reset_index(drop=True)

# Esta línea recupera los datos de geolocalización de los terremotos.
# Selecciona las columnas 'Latitude' y 'Longitude' del DataFrame filtrado usando el indexador loc.
# Similar a la línea anterior, la llamada reset_index(drop=True) se usa para restablecer el índice del DataFrame resultante y descartar el índice original.
earthquake_geolocation_data = df.loc[earthquake_filter, ['Latitude', 'Longitude']].reset_index(drop=True)

# !!! Extraer solo 'Location' de la data geográfica daba error:
# aparecían solo nan en las labels de localidad al hacer hover sobre cada marcador.
# earthquake_geographical_data['Location'] = earthquake_geographical_data['Location'].str.extract(r'Location: (.+)')
# Por este motivo mejor se volvieron a llamar las variables geográficas y de geolocalización y se les implementó reset_index.

# Limpiar data de geolocalización:
# Convertir latitude y longitude a numeric, reemplazando valores no numéricos a NaN.
# Parámetro errors='coerce' para manejar valores no numéricos y convertirlos a NaN.
earthquake_geolocation_data['Latitude'] = pd.to_numeric(earthquake_geolocation_data['Latitude'], errors='coerce')
earthquake_geolocation_data['Longitude'] = pd.to_numeric(earthquake_geolocation_data['Longitude'], errors='coerce')

# !!! Convertir latitude y longitude a strings daba error
# earthquake_geolocation_data['Latitude'] = earthquake_geolocation_data['Latitude'].astype(str)
# earthquake_geolocation_data['Longitude'] = earthquake_geolocation_data['Longitude'].astype(str)

# Borrar filas con valores NaN en latitude y longitude:
# Esta línea descarta cualquier fila en el DataFrame de geolocalización que tenga valores NaN en las columnas 'Latitude' o 'Longitude'
# para garantizar que solo se utilicen datos de geolocalización válidos para trazar en el mapa.
earthquake_geolocation_data.dropna(subset=['Latitude', 'Longitude'], inplace=True)

# Crea un mapa Folium
# centrado en los valores medios de latitud y longitud de los datos de geolocalización del terremoto.
center_lat = earthquake_geolocation_data['Latitude'].mean()
center_lon = earthquake_geolocation_data['Longitude'].mean()
map = folium.Map(location=[center_lat, center_lon], zoom_start=3)

# Bucle for itera sobre cada fila en el DF de datos de geolocalización del terremoto.
# Recupera los valores de latitud y longitud para cada ubicación y crea un marcador en el mapa usando la función folium.Marker.
# Parámetro tooltip configurado para mostrar Location al pasar el cursor sobre marcador.
# El parámetro icon personaliza la apariencia del marcador.
for i, row in earthquake_geolocation_data.iterrows():
    location = earthquake_geographical_data.loc[i, 'Location']
    tooltip = folium.Tooltip(location)
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        tooltip=tooltip,
        icon=folium.Icon(color='orange', icon='exclamation', prefix='fa', icon_size=8)
    ).add_to(map)
pass
# !!! El icono no aparece muy bien al llamar a fontawesome icons para personalizar con icono exclamation.
map

Con respecto al uso de reset_index, se usa en este código para garantizar que el índice de los DataFrames resultantes comience desde 0 y sea continuo sin espacios. No es necesario para el funcionamiento del mapa en sí, pero puede ayudar a indexar y hacer referencia a los datos de manera limpia y coherente, evitando errores.

El uso de reset_index no afecta al DataFrame principal (df) en sí. Crea nuevos DataFrames (earthquake_geographical_data y earthquake_geolocation_data) que contienen los datos filtrados y procesados ​​específicos de los terremotos. Estas nuevas variables están separadas del marco de datos principal y contienen la información relacionada con el terremoto extraída de él.

# Conclusiones

# Recomendaciones

# Guardar dataset en csv

In [9]:
# df.to_csv('../../Data/02ParaLimpiar/02desastres_fillna.csv', index=False, sep=';', encoding='utf-8')

Dataset con valores rellenados y/o filas vacías borradas.